In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')
import torch.nn.init as init
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float32,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float32, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

In [7]:
########### parameter setup
#df = pd.DataFrame()
#df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
niter = 500
rank = 80
tau = 3
gamma = 0.5
rho = 0.5
alpha = 1



print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

loss1 = np.empty(niter)
loss2 = np.empty(niter)
accuracy_train = np.empty(niter)
accuracy_test = np.empty(niter)
time1 = np.empty(niter)

results = torch.zeros(1, 5, niter)



for Out_iter in range(1):
    rank_initial = 400
    seed = 10 + 10*Out_iter
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
    d0 = 561 #561 =3*11*17

    d1 =  1024    # 2^10
    d2 =  1024  
    d3 = 1024
    d4 = 512      # 2^9
    d5 = 512
    d6 = 6 


    W1 = init.uniform_(torch.empty(d1, d0, device=device), a=-0.01, b=0.01)
    W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
    W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
    factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W1_tl_tensor_rec = tt_to_tensor(factors1)
    b1 = 0*torch.ones(d1, 1, device=device) # 0 is stable


    W2 = init.uniform_(torch.empty(d2, d1, device=device), a=-0.01, b=0.01)
    W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
    factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    # we have 9 rank_initial (r1,...,r9) here plus 2 of 1s (r0 & r10).
    # factors: set of tensor cores
    # http://tensorly.org/stable/user_guide/tensor_decomposition.html
    # http://tensorly.org/stable/modules/generated/tensorly.decomposition.tensor_train.html#tensorly.decomposition.tensor_train
    W2_tl_tensor_rec = tt_to_tensor(factors2)
    # tt_to_tensor(factors):Re-assembles ‘factors’, which represent a tensor in TT format into the corresponding full tensor
    #      facros: list of 3d-arrays tt-cores           output_tensor: ndarray      
    b2 = 0*torch.ones(d2, 1, device=device)


    W3 = init.uniform_(torch.empty(d3, d2, device=device), a=-0.01, b=0.01)
    W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
    factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W3_tl_tensor_rec = tt_to_tensor(factors3)
    b3 = 0*torch.ones(d3, 1, device=device)

    W4 = init.uniform_(torch.empty(d4, d3, device=device), a=-0.01, b=0.01)
    W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
    W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W4_tl_tensor_rec = tt_to_tensor(factors4)
    b4 = 0*torch.ones(d4, 1, device=device)

    W5 = init.uniform_(torch.empty(d5, d4, device=device), a=-0.01, b=0.01)
    W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
    W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
    factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
    W5_tl_tensor_rec = tt_to_tensor(factors5)
    b5 = 0*torch.ones(d5, 1, device=device)


    W6 = init.uniform_(torch.empty(d6, d5, device=device), a=-0.01, b=0.01)
    b6 = 0*torch.ones(d6, 1, device=device)

    # W4 = 0.01*torch.randn(d4, d3, device=device)
    # W4_torch_tensor = W4.reshape((50,30,K))
    # W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
    # core4, tucker_factors4 = tucker(W4_tl_tensor, rank=(40,28,K), init='svd', tol=10e-5, random_state=12345)
    # W4_tl_tensor_rec = tl.tucker_to_tensor((core4, tucker_factors4))
    # b4 = 0.1*torch.ones(d4, 1, device=device)

    U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
    V1 = nn.ReLU()(U1)
    U2 = torch.addmm(b2.repeat(1, N), W2, V1)
    V2 = nn.ReLU()(U2)
    U3 = torch.addmm(b3.repeat(1, N), W3, V2)
    V3 = nn.ReLU()(U3)
    U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    V4 = nn.ReLU()(U4)
    U5 = torch.addmm(b5.repeat(1, N), W5, V4)
    V5 = nn.ReLU()(U5)
    U6 = torch.addmm(b6.repeat(1, N), W6, V5)
    V6 = U6 
    # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
    # V4 = U4



    # Iterations
    print('Train on', N, 'samples, validate on', N_test, 'samples')
    for k in range(niter):
        start = time.time()

 # update for last layer
        # update V4
        V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

        # update U4 
        U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

        # update W4 and b4
        W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)
        # W3_torch_tensor = W3.reshape((d3,4,4,4,4,4,4))
        # W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
        # factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,1))
        # W3_tl_tensor_rec = tt_to_tensor(factors3)


        # update for 5th layer
        # update V3
        V5 = updateV(U5,U6,W6,b6,rho,gamma)

        # update U3
        U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

        # update W3 and b3
        W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
        W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W5_tl_tensor_rec = tt_to_tensor(factors5)


  # update for 4th layer
        # update V3
        V4 = updateV(U4,U5,W5,b5,rho,gamma)

        # update U3
        U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

        # update W3 and b3
        W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
        W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W4_tl_tensor_rec = tt_to_tensor(factors4)

  # update for 3nd layer
        # update V3
        V3 = updateV(U3,U4,W4,b4,rho,gamma)

        # update U3
        U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

        # update W3 and b3
        W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
        W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
        factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        #set of tensor cores
        W3_tl_tensor_rec = tt_to_tensor(factors3)

 # update for 2nd layer
        # update V2
        V2 = updateV(U2,U3,W3,b3,rho,gamma)

        # update U2
        U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

        # update W2 and b2
        W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

        # G update (TTD)   
        W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
        W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
        factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W2_tl_tensor_rec = tt_to_tensor(factors2)


# update for 1st layer
        # update V1
        V1 = updateV(U1,U2,W2,b2,rho,gamma)

        # update U1
        U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

        # update W1 and b1
        W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

        # G update
        W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
        W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
        factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
        W1_tl_tensor_rec = tt_to_tensor(factors1)




 # prediction for trainning data
        a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_train))
        #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train))
        a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_train))
        a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_train))
        #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
        # check argmax and addmm, dim=0

 #Prediction for test data
        a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), X_test))
        #a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test))
        a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).float(), a3_test))
        a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).float(), a4_test))
        pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_train))
        # # a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_train))
        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).float(), x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).float(), a1_test))
        # # a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)), a2_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).float(), a2_test), dim=0)

        # a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
        # a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), W2, a1_train))

        # #print(torch.addmm(b4.repeat(1, N), W4, a3_train))
        # pred = torch.argmax(torch.addmm(b3.repeat(1, N), W3, a2_train), dim=0)

        # a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), W1, x_test))
        # a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), W2, a1_test))
        # pred_test = torch.argmax(torch.addmm(b3.repeat(1, N_test), W3, a2_test), dim=0)




   #emperical loss
        loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
        # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
        # torch.disk: Returns the p-norm of (input - other)
        # Eq (5) in paper
        loss2[k] = loss1[k] + rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() \
        +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() \
        + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \
        +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() \

        # +tau/2*torch.pow(torch.dist(W3.reshape((d3,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).float(),2),2).cpu().numpy() 
        # +tau/2*torch.pow(torch.dist(W4.reshape((50,30,K)),torch.as_tensor(W4_tl_tensor_rec,device=device),2),2).cpu().numpy() 

        # compute training accuracy
        correct_train = pred == y_train-1
        accuracy_train[k] = np.mean(correct_train.cpu().numpy())

        # compute validation accuracy
        correct_test = pred_test == y_test-1
        accuracy_test[k] = np.mean(correct_test.cpu().numpy())

        # compute training time
        stop = time.time()
        duration = stop - start
        time1[k] = duration

        # print results
        print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
              '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
              '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

##############
############## compute CR
    factors1_shape=[f.shape for f in factors1]
    Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
    factors2_shape=[f.shape for f in factors2]
    Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
    factors3_shape=[f.shape for f in factors3]
    Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
    factors4_shape=[f.shape for f in factors4]
    Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
    factors5_shape=[f.shape for f in factors5]
    Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

    total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

    CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
    CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
    print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


    results[Out_iter,0,:] = torch.tensor(loss1)
    results[Out_iter,1,:] = torch.tensor(loss2)
    results[Out_iter,2,:] = torch.tensor(accuracy_train)
    results[Out_iter,3,:] = torch.tensor(accuracy_test)
    results[Out_iter,4,:] = torch.tensor(time1)
    CR=(CR_1,CR_2)

#     #this postion to add new row into existing table
#         df=pd.read_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv')
#         new_row = {'rank':rank, 'CR_1':CR_1, 'CR_2':CR_2, 'tau':tau, 'gamma':gamma, 'rho':rho, 'alpha':alpha, \
#                    'loss1':loss1[niter-1], 'loss2':loss2[niter-1], 'accuracy_train':accuracy_train[niter-1], \
#                    'accuracy_test':accuracy_test[niter-1], 'time':time1[niter-1],'seed':seed} 
#         df=df.append(new_row,ignore_index=True)
#         df.to_csv('C:/Users/Mark/Desktop/HAR_5layers_rank40_100times.csv',index=False)


filename= "UniformScaled_" + "niter_"+ str(niter) + "rank_" + str(rank) + "tau_" + str(tau) + "gamma_" + str(gamma) + \
"rho_" + str(rho) + "alpha_" + str(alpha) + ".mat"
from scipy.io import savemat
%cd /home/c/cl237/TenBCD/UCI HAR/5 hidden layers/Different Initialization/
savemat (filename, {'results': torch.Tensor.numpy(results), 'Compression Ratio':CR})
#this position to save table into matlab
#df.to_csv('C:/Users/Mark/Desktop/result_compress_123_layer_highrank.csv')
##Changing Folder
 #%cd '/content/gdrive/MyDrive/Colab/Tensor-BCD-for-DNN/Experiments/DataSaved'
 #savemat("ThreeLayer_rank160.mat", {'results': torch.Tensor.numpy(results)})    

rank= 80 tau= 3 gamma= 0.5 rho= 0.5 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 3.813204526901245 - sq_loss: 661.678466796875 - tot_loss: 924.9924528305496 - acc: 0.17859085963003266 - val_acc: 0.168985408890397
Repeatition 1 Epoch 2 / 500 
 - time: 3.1159539222717285 - sq_loss: 294.0793151855469 - tot_loss: 484.89320694527123 - acc: 0.20797062023939064 - val_acc: 0.19239904988123516
Repeatition 1 Epoch 3 / 500 
 - time: 3.0232977867126465 - sq_loss: 165.0137481689453 - tot_loss: 264.86316735250875 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 4 / 500 
 - time: 3.1198904514312744 - sq_loss: 90.42965698242188 - tot_loss: 148.2673159246333 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 5 / 500 
 - time: 3.0823814868927 - sq_loss: 48.922645568847656 - tot_loss: 85.64182902732864 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 6 / 500 
 - time: 3.08201456069

Repeatition 1 Epoch 48 / 500 
 - time: 3.1245858669281006 - sq_loss: 0.00045608493383042514 - tot_loss: 1.1575935223700071 - acc: 0.5346844396082698 - val_acc: 0.5212080081438751
Repeatition 1 Epoch 49 / 500 
 - time: 3.071181535720825 - sq_loss: 0.0004245070740580559 - tot_loss: 1.1134993216082876 - acc: 0.5412132752992383 - val_acc: 0.5269765863590091
Repeatition 1 Epoch 50 / 500 
 - time: 3.098316192626953 - sq_loss: 0.0003957573208026588 - tot_loss: 1.0791120416115518 - acc: 0.5432535364526659 - val_acc: 0.5279945707499152
Repeatition 1 Epoch 51 / 500 
 - time: 3.044724702835083 - sq_loss: 0.00036944885505363345 - tot_loss: 1.0419405997072317 - acc: 0.5457018498367792 - val_acc: 0.5293518832711231
Repeatition 1 Epoch 52 / 500 
 - time: 3.0877580642700195 - sq_loss: 0.00034519191831350327 - tot_loss: 1.0074506321043373 - acc: 0.5471980413492927 - val_acc: 0.5307091957923312
Repeatition 1 Epoch 53 / 500 
 - time: 3.1394636631011963 - sq_loss: 0.0003228900895919651 - tot_loss: 0.97778

Repeatition 1 Epoch 95 / 500 
 - time: 3.086115837097168 - sq_loss: 2.961941390822176e-05 - tot_loss: 0.2988174970384989 - acc: 0.7825081610446137 - val_acc: 0.7573803868340685
Repeatition 1 Epoch 96 / 500 
 - time: 3.0095508098602295 - sq_loss: 2.8071875931345858e-05 - tot_loss: 0.2869498829037411 - acc: 0.7834602829162133 - val_acc: 0.7587376993552766
Repeatition 1 Epoch 97 / 500 
 - time: 3.0457394123077393 - sq_loss: 2.6609692213241942e-05 - tot_loss: 0.28079263228573836 - acc: 0.7842763873775843 - val_acc: 0.7607736681370886
Repeatition 1 Epoch 98 / 500 
 - time: 3.079810380935669 - sq_loss: 2.525557101762388e-05 - tot_loss: 0.27110584829188156 - acc: 0.7864526659412405 - val_acc: 0.7628096369189006
Repeatition 1 Epoch 99 / 500 
 - time: 3.060966730117798 - sq_loss: 2.4005514205782674e-05 - tot_loss: 0.2710176506905668 - acc: 0.7880848748639826 - val_acc: 0.7651849338310146
Repeatition 1 Epoch 100 / 500 
 - time: 3.097782611846924 - sq_loss: 2.2823780454928055e-05 - tot_loss: 0.25

Repeatition 1 Epoch 141 / 500 
 - time: 3.076361894607544 - sq_loss: 3.5918064895668067e-06 - tot_loss: 0.1502379266232623 - acc: 0.8807127312295974 - val_acc: 0.8520529351883271
Repeatition 1 Epoch 142 / 500 
 - time: 3.013364315032959 - sq_loss: 3.4658328331715893e-06 - tot_loss: 0.14175761384976937 - acc: 0.8834330794341676 - val_acc: 0.8527315914489311
Repeatition 1 Epoch 143 / 500 
 - time: 3.0745127201080322 - sq_loss: 3.3461462862760527e-06 - tot_loss: 0.13563772136882335 - acc: 0.8852013057671382 - val_acc: 0.8544282321004412
Repeatition 1 Epoch 144 / 500 
 - time: 3.069274663925171 - sq_loss: 3.23351014230866e-06 - tot_loss: 0.1342105197650838 - acc: 0.8872415669205659 - val_acc: 0.8561248727519511
Repeatition 1 Epoch 145 / 500 
 - time: 3.081641912460327 - sq_loss: 3.1260353807738284e-06 - tot_loss: 0.13725499896173687 - acc: 0.8883297062023939 - val_acc: 0.8588394977943672
Repeatition 1 Epoch 146 / 500 
 - time: 3.0486550331115723 - sq_loss: 3.0223113753891084e-06 - tot_loss

Repeatition 1 Epoch 187 / 500 
 - time: 3.11938214302063 - sq_loss: 1.162442572422151e-06 - tot_loss: 0.10118722237482292 - acc: 0.920157780195865 - val_acc: 0.9080420766881574
Repeatition 1 Epoch 188 / 500 
 - time: 3.0696451663970947 - sq_loss: 1.1500707159939338e-06 - tot_loss: 0.10362536070274153 - acc: 0.9207018498367792 - val_acc: 0.9083814048184594
Repeatition 1 Epoch 189 / 500 
 - time: 3.044933795928955 - sq_loss: 1.1338103149682865e-06 - tot_loss: 0.10214828812875076 - acc: 0.9211099020674647 - val_acc: 0.9080420766881574
Repeatition 1 Epoch 190 / 500 
 - time: 3.0620875358581543 - sq_loss: 1.1211524224563618e-06 - tot_loss: 0.10454497925352157 - acc: 0.9216539717083787 - val_acc: 0.9087207329487614
Repeatition 1 Epoch 191 / 500 
 - time: 3.031355381011963 - sq_loss: 1.1069687388953753e-06 - tot_loss: 0.099649397682394 - acc: 0.9219260065288357 - val_acc: 0.9083814048184594
Repeatition 1 Epoch 192 / 500 
 - time: 3.037149429321289 - sq_loss: 1.090589080376958e-06 - tot_loss: 

...

Repeatition 1 Epoch 279 / 500 
 - time: 3.1050846576690674 - sq_loss: 6.346925829348038e-07 - tot_loss: 0.08266264709800386 - acc: 0.9530739934711643 - val_acc: 0.9375636240244316
Repeatition 1 Epoch 280 / 500 
 - time: 3.0612995624542236 - sq_loss: 6.327299502117967e-07 - tot_loss: 0.08372870804388843 - acc: 0.9533460282916213 - val_acc: 0.9389209365456397
Repeatition 1 Epoch 281 / 500 
 - time: 3.1342978477478027 - sq_loss: 6.302470865193754e-07 - tot_loss: 0.08520303514719219 - acc: 0.9537540805223068 - val_acc: 0.9389209365456397
Repeatition 1 Epoch 282 / 500 
 - time: 3.0214083194732666 - sq_loss: 6.281211994974001e-07 - tot_loss: 0.07638260197495206 - acc: 0.9540261153427638 - val_acc: 0.9389209365456397
Repeatition 1 Epoch 283 / 500 
 - time: 3.0236902236938477 - sq_loss: 6.263351224333746e-07 - tot_loss: 0.08145075991409478 - acc: 0.954842219804135 - val_acc: 0.9392602646759416
Repeatition 1 Epoch 284 / 500 
 - time: 3.0457651615142822 - sq_loss: 6.245130634852103e-07 - tot_los

Repeatition 1 Epoch 325 / 500 
 - time: 3.125966787338257 - sq_loss: 5.602767032542033e-07 - tot_loss: 0.08493162353220973 - acc: 0.9639553862894451 - val_acc: 0.9443501866304717
Repeatition 1 Epoch 326 / 500 
 - time: 3.12489652633667 - sq_loss: 5.59447812520375e-07 - tot_loss: 0.08157952811280378 - acc: 0.9644994559303591 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 327 / 500 
 - time: 3.0447003841400146 - sq_loss: 5.5665776699243e-07 - tot_loss: 0.07935407565858155 - acc: 0.9647714907508161 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 328 / 500 
 - time: 3.114706516265869 - sq_loss: 5.534506044568843e-07 - tot_loss: 0.08079540912051808 - acc: 0.9650435255712732 - val_acc: 0.9453681710213777
Repeatition 1 Epoch 329 / 500 
 - time: 3.068784236907959 - sq_loss: 5.529192321773735e-07 - tot_loss: 0.08653556179563116 - acc: 0.9650435255712732 - val_acc: 0.9463861554122837
Repeatition 1 Epoch 330 / 500 
 - time: 3.0340898036956787 - sq_loss: 5.52872450043651e-07 - tot_loss: 0.077

Repeatition 1 Epoch 371 / 500 
 - time: 3.111637830734253 - sq_loss: 5.065745654064813e-07 - tot_loss: 0.07879996079536744 - acc: 0.9717083786724701 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 372 / 500 
 - time: 3.0563652515411377 - sq_loss: 5.064172228230746e-07 - tot_loss: 0.08537849981479984 - acc: 0.9718443960826986 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 373 / 500 
 - time: 3.0907142162323 - sq_loss: 5.064362653683929e-07 - tot_loss: 0.08280633392143844 - acc: 0.9719804134929271 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 374 / 500 
 - time: 3.1219077110290527 - sq_loss: 5.05834691466589e-07 - tot_loss: 0.08532945124816238 - acc: 0.9722524483133841 - val_acc: 0.9511367492365117
Repeatition 1 Epoch 375 / 500 
 - time: 2.949526071548462 - sq_loss: 5.054558869233006e-07 - tot_loss: 0.08262117308356032 - acc: 0.9723884657236126 - val_acc: 0.9524940617577197
Repeatition 1 Epoch 376 / 500 
 - time: 3.0555124282836914 - sq_loss: 5.03486489833449e-07 - tot_loss: 0.0

Repeatition 1 Epoch 417 / 500 
 - time: 1.4489727020263672 - sq_loss: 4.6987273094600823e-07 - tot_loss: 0.08250571192417422 - acc: 0.9764689880304679 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 418 / 500 
 - time: 1.4527034759521484 - sq_loss: 4.6851607748976676e-07 - tot_loss: 0.08045929234570726 - acc: 0.9764689880304679 - val_acc: 0.9548693586698337
Repeatition 1 Epoch 419 / 500 
 - time: 1.4435968399047852 - sq_loss: 4.6708458967259503e-07 - tot_loss: 0.09128972269772728 - acc: 0.9766050054406964 - val_acc: 0.9541907024092298
Repeatition 1 Epoch 420 / 500 
 - time: 1.4488928318023682 - sq_loss: 4.674058402542869e-07 - tot_loss: 0.07568320114624949 - acc: 0.9768770402611534 - val_acc: 0.9545300305395318
Repeatition 1 Epoch 421 / 500 
 - time: 1.4439127445220947 - sq_loss: 4.671597935157479e-07 - tot_loss: 0.07971107785566922 - acc: 0.9770130576713819 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 422 / 500 
 - time: 1.4413845539093018 - sq_loss: 4.672446038966882e-07 - tot

Repeatition 1 Epoch 463 / 500 
 - time: 1.4622740745544434 - sq_loss: 4.3893999190913746e-07 - tot_loss: 0.07905424248258697 - acc: 0.9791893362350381 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 464 / 500 
 - time: 1.4548029899597168 - sq_loss: 4.388172953895264e-07 - tot_loss: 0.08533859474464878 - acc: 0.9793253536452666 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 465 / 500 
 - time: 1.4559552669525146 - sq_loss: 4.384486089747952e-07 - tot_loss: 0.0755424268390239 - acc: 0.9794613710554951 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 466 / 500 
 - time: 1.4613797664642334 - sq_loss: 4.367579151676182e-07 - tot_loss: 0.08403651878910312 - acc: 0.9794613710554951 - val_acc: 0.9552086868001357
Repeatition 1 Epoch 467 / 500 
 - time: 1.4735782146453857 - sq_loss: 4.350875428826839e-07 - tot_loss: 0.08711554178535197 - acc: 0.9795973884657236 - val_acc: 0.9555480149304377
Repeatition 1 Epoch 468 / 500 
 - time: 1.4670209884643555 - sq_loss: 4.333104755005479e-07 - tot_lo

In [8]:
seed

10